<a href="https://colab.research.google.com/github/UmarKhattab09/Model-Context-Protocol/blob/main/MCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import pipeline


In [2]:
from transformers import AutoModelForCausalLM
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_id = "meta-llama/Llama-3.2-3B-Instruct"
from google.colab import userdata
token=userdata.get('hf_token')
# model = AutoModelForSequenceClassification.from_pretrained(model_id, use_auth_token=token)
model = AutoModelForCausalLM.from_pretrained(model_id,device_map="cuda",torch_dtype="auto",trust_remote_code=True,use_auth_token=token)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=token)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [3]:
pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:
messages=[
    {"role":"user","content":"Can you provide me instructions to create a cake"}
]
generate_args={
    "max_new_tokens": 2000,
    "return_full_text":False,
    "do_sample":False #Randomnize
}

In [5]:
output = pipe(messages,**generate_args)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
print(output[0]['generated_text'])

Here's a step-by-step guide to creating a basic cake:

**Ingredients:**

For a classic vanilla cake:

* 2 cups all-purpose flour
* 1 teaspoon baking powder
* 1 teaspoon salt
* 1 cup granulated sugar
* 1/2 cup unsalted butter, softened
* 2 large eggs
* 2 teaspoons vanilla extract
* 1 cup whole milk, at room temperature

For the frosting (optional):

* 1 cup unsalted butter, softened
* 2 cups powdered sugar
* 1 teaspoon vanilla extract
* 2-4 tablespoons milk or heavy cream

**Equipment:**

* 9-inch round cake pan
* Non-stick cooking spray or parchment paper
* Electric mixer (stand or handheld)
* Whisk
* Measuring cups and spoons
* Rubber spatula

**Instructions:**

**Step 1: Preheat and Prepare the Pan**

1. Preheat your oven to 350°F (180°C).
2. Spray the 9-inch round cake pan with non-stick cooking spray or line it with parchment paper.

**Step 2: Mix the Dry Ingredients**

1. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.

**Step 3: Mix the Wet Ingredi

## RESPONNSE FUNCTION

In [7]:
def get_response_from_message_hf(pipe,messages,temperature=0):
  sampling = False if temperature == 0 else True

  generate_args={
    "max_new_tokens": 2000,
    "return_full_text":False,
    "do_sample":sampling  #Randomnize
    }
  if temperature != 0:
    generate_args['temperature']=temperature

  output=pipe(messages,**generate_args)
  return output[0]['generated_text']


In [8]:
#PROMPT ENGINEERING
messages=[
    {"role":"system","content":"You are a Helpful AI assistant for a user who's name is ben"},
    {"role":"user","content":"what is my name"}
]

In [9]:
output=get_response_from_message_hf(pipe,messages)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [10]:
output

'Your name is Ben.'

In [11]:
systemprompt = """
You are a helpful AI assistant that classifies customer support messages into one of those 3 categories
1. Technical Issue
2. Billing Industry
3. General Feedback
"""
user_input = "I love this Product"

messages = [
 {"role":"system","content": systemprompt},
  {"role":"user","content":user_input}
            ]

In [12]:
output = get_response_from_message_hf(pipe,messages)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [13]:
output

'I would classify your message as: General Feedback'

In [14]:
def send_to_general_feedback(user_message):
  print("sent to general feedback")

def send_to_billing_industry(user_message):
  print("sent to billing industry")

def send_to_technical_issue(user_message):
  print("sent to technical issue")

In [15]:
if "General Feedback" in output:
  send_to_general_feedback(user_input)
if "Billing Industry" in output:
  send_to_billing_industry(user_input)
if "Technical Issue" in output:
  send_to_technical_issue(user_input)

sent to general feedback


In [16]:
#EXAMPLE OF OUTPUT WHICH IS HARDER TO DETECT
# OUT OF Technical Issue, Billing Inquiry and General Feedback, I classify it as General Feedback

In [17]:
systemprompt = """
You are a helpful AI assistant that classifies customer support messages into one of those 3 categories
1. Technical Issue
2. Billing Industry
3. General Feedback

the output should be in a json format like this
{
  "category":"use one from the 3 categories above"
}
"""
user_input = "I love this Product"

messages = [
 {"role":"system","content": systemprompt},
  {"role":"user","content":user_input}
            ]

In [18]:
output = get_response_from_message_hf(pipe,messages)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [19]:
import json
op=json.loads(output)
op['category']

'General Feedback'

In [20]:
if op['category'] == "General Feedback":
  send_to_general_feedback(user_input)
if op['category'] == "Billing Industry":
  send_to_billing_industry(user_input)
if op['category'] == "Technical Issue":
  send_to_technical_issue(user_input)

sent to general feedback


In [21]:
function_dict = {
    "General Feedback":send_to_general_feedback,
    "Billing Industry":send_to_billing_industry,
    "Technical Issue":send_to_technical_issue
}
function_dict[op['category']](user_input)

sent to general feedback


## MCP CLIENT STEPS:
- Know what functions are available to use
- Determining if the user proompts requires the output of one of those function, this is call prompt intent
- lastly run the function and append its output to the llm

In [22]:
## Prompt Intent
## 1. TASK
## 2. Classifictions
## 3. Output Form

### MCP DEVELOPMENT

In [23]:
def get_user_name():
  return "umar"

def get_user_age():
  return 22

In [24]:
tools={
    "get_user_name": {
        "description": "This tools returns the name of the user",
        "tool": get_user_name,
        "parameters": None

    },
    "get_user_age": {
        "description": "This tools returns the age of the user",
        "tool": get_user_age,
        "parameters": None,
    }}

In [25]:
tools_str = str(tools)

In [26]:

system_prompt= f"""
You are a system that reads each user message and decides whether to invoke one of the available tools to generate the answer.
Not every message requires a tool. Only select a tool when it directly helps respond to the user’s request.

The full list of available tools is provided as JSON here:
{tools_str}

When you respond, follow this pattern:
  1.
      Reason whether any of the provided tools is a good fit or not.
      Reasoning should be 2 sentences or less
      Reasoning should end with a new line then "no tool is needed" If no tool is a good fit or if no tools are relevant.
  2.
    On its own line, write: "Json Output"
  3.
    Choose one the following two options:
    a. If you determine no tool is needed, simply return an empty json like so {{}}
    b. If a tool is needed: provide a JSON object with exactly two keys like the one bellow:
      {{
        "tool_name":"" # put here the name of the tool,
        "parameters":{{}} # parameters in a proper json format, this is going to be empty if there are no parameters.
      }}
"""

user_message = "What's my name?"

messages =[
    {"role":"system","content":system_prompt},
    {"role":"user","content":user_message}
    ]


In [27]:
output = get_response_from_message_hf(pipe,messages)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [28]:
print(output)

1. The 'get_user_name' tool is a good fit for this request as it directly returns the name of the user.
 
Json Output

{
  "tool_name": "get_user_name",
  "parameters": {}
}


In [29]:
messages =[
    {"role":"system","content":system_prompt},
    {"role":"user","content":"Capital of France?"}
    ]
output = get_response_from_message_hf(pipe,messages)
print(output)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1. The 'get_user_name' tool is not a good fit for this request as it is not relevant to finding the capital of France. The 'get_user_age' tool is also not a good fit for this request for the same reason.

Json Output

{
  "tool_name": "get_user_name",
  "parameters": {}
}


In [30]:
messages =[
    {"role":"system","content":system_prompt},
    {"role":"user","content":"What is my age?"}
    ]
output = get_response_from_message_hf(pipe,messages)
print(output)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1. The 'get_user_age' tool is a good fit for this request because it directly returns the user's age, which is the information the user is asking for. This tool is the most relevant to the user's query.

Json Output

{
  "tool_name": "get_user_age",
  "parameters": {}
}


In [31]:


def get_message_intent(pipe,tools,user_message):
    tool_list_str = str(tools)

    system_prompt = f"""
    You are a system that reads each user message and decides whether to invoke one of the available tools to generate the answer.
    Not every message requires a tool.  Only select a tool when it directly helps respond to the user’s request.

    The full list of available tools is provided as JSON here:
    {tool_list_str}

    When you respond, follow this pattern:
    1.
        Reason whether any of the provded tools is a good fit or not.
        Reasoning should be 2 sentences or less
        Reasoning should end with a new line then "no tool is needed" If no tool is a good fit or if no tools are relevant.
    2.
      On its own line, write: "Json Output"
    3.
      Choose one the following two options:
      a. If you determine no tool is needed, simply return an empty json like so {{}}
      b. If a tool is needed: provide a JSON object with exactly two keys like the one bellow:
        {{
          "tool_name":"" # put here the name of the tool,
          "parameters":{{}} # parameters in a proper json format, this is going to be empty if there are no parameters.
        }}

    don't number the output 1,2,3 or a,b.
    """

    messages =[
      {"role":"system","content":system_prompt},
      {"role":"user","content":user_message}
    ]

    output = get_response_from_message_hf(pipe,messages)

    return output

In [32]:
output = get_message_intent(pipe,tools,"What's my name?")
print(output)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Reason: The 'get_user_name' tool is a good fit for this request as it directly returns the name of the user. This tool is relevant to the user's request.

Json Output

{
  "tool_name": "get_user_name",
  "parameters": {}
}


In [33]:
json_output = json.loads(((output.lower().split('json output')[1]).strip()))
json_output

{'tool_name': 'get_user_name', 'parameters': {}}

In [34]:
function_to_run=(json_output['tool_name'])
parameters = json_output['parameters']

In [35]:
tools

{'get_user_name': {'description': 'This tools returns the name of the user',
  'tool': <function __main__.get_user_name()>,
  'parameters': None},
 'get_user_age': {'description': 'This tools returns the age of the user',
  'tool': <function __main__.get_user_age()>,
  'parameters': None}}

In [36]:
result = tools[function_to_run]['tool'](**parameters)
result

'umar'

In [37]:
tool_run_output = f"""This is output when we ran {function_to_run} that does this {tools[function_to_run]['description']}
    Tool Output: {result}
    """
print(tool_run_output)

This is output when we ran get_user_name that does this This tools returns the name of the user
    Tool Output: umar
    


In [38]:
user_message = "Whats my name"
messages = [
    {"role":"user","content":user_message+"""
    That tool’s output will be:
        Data Output – Information you should incorporate into your answer as factual content.

      When you compose your reply, follow these steps:

      1. Inspect the tool output
          weave it into your response as part of your knowledge

      2. Respond appropriately
         use it to directly inform or support your answer.

    """+tool_run_output}
    ]

In [39]:
user_facing_response = get_response_from_message_hf(pipe,messages)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [40]:
print(user_facing_response)

Based on the tool output, it appears that your name is Umar.


In [41]:
user_message = "Capital of France?"
messages = [
    {"role":"user","content":user_message+"""
    That tool’s output will be:
        Data Output – Information you should incorporate into your answer as factual content.

      When you compose your reply, follow these steps:

      1. Inspect the tool output
          weave it into your response as part of your knowledge

      2. Respond appropriately
         use it to directly inform or support your answer.

    """+tool_run_output}
    ]
user_facing_response = get_response_from_message_hf(pipe,messages)
print(user_facing_response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The capital of France is Paris.


In [42]:
def run_tools(tools,message_intent):
  json_output = json.loads(((message_intent.lower().split('json output')[1]).strip()))

  if len(json_output)==0:
    return ""

  function_to_run = json_output['tool_name']
  parameters = json_output['parameters']
  result = tools[function_to_run]["tool"](**parameters)

  output_response = f"""This is output when we ran {function_to_run} that does this {tools[function_to_run]['description']}
    Tool Output: {result}
    """

  return output_response

In [43]:
output

'Reason: The \'get_user_name\' tool is a good fit for this request as it directly returns the name of the user. This tool is relevant to the user\'s request.\n\nJson Output\n\n{\n  "tool_name": "get_user_name",\n  "parameters": {}\n}'

In [45]:
tool_run_output = run_tools(tools,output)
print(tool_run_output)

This is output when we ran get_user_name that does this This tools returns the name of the user
    Tool Output: umar
    


In [47]:
class Client:
  def __init__(self,tools):
    self.tools=tools
    self.strtools = str(tools)



  def get_message_intent(self,pipe,user_message):
    system_prompt = f"""
      You are a system that reads each user message and decides whether to invoke one of the available tools to generate the answer.
      Not every message requires a tool.  Only select a tool when it directly helps respond to the user’s request.

      The full list of available tools is provided as JSON here:
        {self.strtools}

      When you respond, follow this pattern:
      1.
          Reason whether any of the provded tools is a good fit or not.
          Reasoning should be 2 sentences or less
          Reasoning should end with a new line then "no tool is needed" If no tool is a good fit or if no tools are relevant.
      2.
        On its own line, write: "Json Output"
      3.
        Choose one the following two options:
        a. If you determine no tool is needed, simply return an empty json like so {{}}
        b. If a tool is needed: provide a JSON object with exactly two keys like the one bellow:
          {{
            "tool_name":"" # put here the name of the tool,
            "parameters":{{}} # parameters in a proper json format, this is going to be empty if there are no parameters.
          }}

      don't number the output 1,2,3 or a,b.
      """
    messages =[
        {"role":"system","content":system_prompt},
        {"role":"user","content":user_message}
      ]

    output = get_response_from_message_hf(pipe,messages)

    return output


  def run_tools(self,message_intent):
    json_output = json.loads(((message_intent.lower().split('json output')[1]).strip()))

    if len(json_output)==0:
      return ""

    function_to_run = json_output['tool_name']
    parameters = json_output['parameters']
    result = self.tools[function_to_run]["tool"](**parameters)

    output_response = f"""This is output when we ran {function_to_run} that does this {tools[function_to_run]['description']}
      Tool Output: {result}
      """

    return output_response


In [48]:
# Host
class Host():
  def __init__(self,llm_model,client):
    self.client = client
    self.llm_model=llm_model
    self.message_history=[]

  def get_user_response(self,user_message):
    original_user_message= user_message

    intent_output = self.client.get_message_intent(self.llm_model,user_message)
    tool_output = self.client.run_tools(intent_output)


    if tool_output.strip() !="" and  tool_output.strip() !="no tool is needed":
      message_tool_attachement = """
      That tool’s output will be one of two kinds:
        1. Data Output – Information you should incorporate into your answer as factual content.
        2. Status Output – A message indicating success or failure of the tool or an opperation. In this case, simply report the status to the user; do not suggest or describe how to run the tool.

      When you compose your reply, follow these steps:

      1. Inspect the tool output
          a. If it’s data, weave it into your response as part of your knowledge.
          b. If it’s a function status message, communicate that status to the user.

      2. Respond appropriately
        a. For data: use it to directly inform or support your answer.
        b. For function status: acknowledge success or explain the error without offering tool-invocation instructions.

      """+tool_output

      user_message+= message_tool_attachement

    self.message_history.append({"role":"user","content":user_message})

    user_facing_output = get_response_from_message_hf(self.llm_model,self.message_history)

    self.message_history[-1]['content'] = original_user_message
    self.message_history.append({"role":"assistant","content":user_facing_output})
    return user_facing_output


In [49]:
# Resources
def get_user_name():
  return "Ben"
def get_user_age():
  return 28

# tool
def write_to_file(file_path,content):
  with open(file_path,'w') as f:
    f.write(content)
  return "File written successfully in the specified path: "+file_path

In [50]:
# Student Database
student_database_json = {
    "s001": {
        "name": "Alice",
        "GPA": 3.2,
        "major": "Computer Science",
        "Courses" : ["programming 101","networks","AI", "digital logic"]
    },
    "s002": {
        "name": "Bob",
        "GPA": 3.8,
        "major": "Mathematics",
        "Courses" : ["programming 101","math 101","english 201", "linear algebra"]
    }
}

def get_student_gpa(student_id):
  if student_id in student_database_json:
    return student_database_json[student_id]["GPA"]
  else:
    return "Student not found"

def get_student_major(student_id):
  if student_id in student_database_json:
    return student_database_json[student_id]["major"]
  else:
    return "Student not found"

def get_student_courses(student_id):
  if student_id in student_database_json:
    return student_database_json[student_id]["Courses"]
  else:
    return "Student not found"

def add_student(student_id,student_name,student_gpa,student_major,student_courses):
  student_database_json[student_id] = {
      "name": student_name,
      "GPA": student_gpa,
      "major": student_major,
      "Courses" : student_courses }

  return "Student added successfully"

def edit_student_gpa(student_id,new_gpa):
  if student_id in student_database_json:
    student_database_json[student_id]["GPA"] = new_gpa
    return "GPA updated successfully"
  else:
    return "Student not found"


In [51]:
tools = {
    "get_user_name": {
        "description": "This tool returns the user's name",
        "tool":get_user_name,
        "parameters": None,
    },
    "get_user_age": {
        "description": "This tool returns the user's age",
        "tool":get_user_age,
        "parameters":None,
    },
    "write_to_file": {
        "description": "This tool writes content to a file",
        "tool":write_to_file,
        "parameters":{'file_path':'the path where the file should be saved', 'content':'the content that should be written in the file'},
    },

    "get_student_gpa": {
        "description": "This tool returns the student's GPA",
        "tool":get_student_gpa,
        "parameters":{'student_id':'the id of the student'},
    },
    "get_student_major": {
        "description": "This tool returns the student's major",
        "tool":get_student_major,
        "parameters":{'student_id':'the id of the student'},
    },
    "get_student_courses": {
        "description": "This tool returns the student's courses",
        "tool":get_student_courses,
        "parameters":{'student_id':'the id of the student'},
    },
    "add_student": {
        "description": "This tool adds a student to the database",
        "tool":add_student,
        "parameters":{'student_id':'the id of the student', 'student_name':'the name of the student', 'student_gpa':'the GPA of the student', 'student_major':'the major of the student', 'student_courses':'the courses of the student'},
    },
    "edit_student_gpa": {
        "description": "This tool edits the student's GPA",
        "tool":edit_student_gpa,
        "parameters":{'student_id':'the id of the student', 'new_gpa':'the new GPA of the student'},
    }
}

In [52]:
client = Client(tools)
host = Host(pipe,client)

In [53]:
user_facing_output = host.get_user_response("What's my name?")
print(user_facing_output)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Your name is Ben.


In [54]:
user_facing_output = host.get_user_response("Write 'hello there' in a text file in this path 'greetings.txt' ")
print(user_facing_output)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The tool output indicates that the file was written successfully. Here's a response:

The file "greetings.txt" has been written to the specified path. The content of the file is "hello there".


In [55]:
user_facing_output = host.get_user_response("What's the major of the student with id s002?")
print(user_facing_output)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The major of the student with id s002 is Mathematics.


In [58]:
user_facing_output = host.get_user_response("Edit the GPA of student s001 to 3.8")
print(user_facing_output)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The GPA of student s001 has been updated to 3.8.


In [59]:
student_database_json['s001']

{'name': 'Alice',
 'GPA': '3.8',
 'major': 'Computer Science',
 'Courses': ['programming 101', 'networks', 'AI', 'digital logic']}